In [1]:
import os
import csv
import numpy as np
import supervision as sv
from ultralytics import YOLO

In [2]:
# --- Setup paths and parameters ---
INPUT_VIDEO_PATH = r"E:/Random Python Scripts/Tata HaxS/supervision/input/dented_car_video_720p.mp4"  # Path to your input video
OUTPUT_VIDEO_PATH = os.path.join(os.getcwd(), "results.mp4")  # Output video path
CSV_OUTPUT_PATH = os.path.join(os.getcwd(), "tracking_results.csv")
MODEL_PATH = r"E:/Random Python Scripts/Tata HaxS/supervision/model/best.pt"  # Path to your custom YOLOv8 weights file

In [3]:
# --- Load YOLOv8 model and set up classes ---
model = YOLO(MODEL_PATH)
CLASS_NAMES_DICT = model.model.names  # e.g., {0: "dent", 1: "scratch"}

In [4]:
# Only count our selected classes
SELECTED_CLASSES = ['dent', 'scratch']
SELECTED_CLASS_IDS = [class_id for class_id, name in CLASS_NAMES_DICT.items() if name in SELECTED_CLASSES]

In [5]:
# --- Initialize the ByteTracker ---
byte_tracker = sv.ByteTrack(
    track_activation_threshold=0.25,
    lost_track_buffer=30,
    minimum_matching_threshold=0.8,
    frame_rate=30,
    minimum_consecutive_frames=3
)

In [6]:
# --- Global dictionaries for unique counts and tracking records ---
unique_counts = {class_name: set() for class_name in SELECTED_CLASSES}
# # tracking_records will store one record per detection (one row per detection)
# tracking_records = []

In [7]:
# --- Create the frame generator ---
generator = sv.get_video_frames_generator(INPUT_VIDEO_PATH)

In [8]:
# --- Define the callback function for processing frames ---
def callback(frame: np.ndarray, index: int) -> np.ndarray:
    # Run YOLO inference on the current frame.
    results = model(frame, verbose=False)[0]
    detections = sv.Detections.from_ultralytics(results)
    
    # Filter detections to only include our selected classes.
    mask = np.isin(detections.class_id, SELECTED_CLASS_IDS)
    detections = detections[mask]
    
    # Update the tracker with these detections.
    detections = byte_tracker.update_with_detections(detections)
    
    # For each detection, add its tracker ID to the overall unique count.
    for conf, class_id, tracker_id in zip(detections.confidence, detections.class_id, detections.tracker_id):
        if tracker_id is not None:
            class_name = CLASS_NAMES_DICT[class_id]
            unique_counts[class_name].add(tracker_id)
    
    # (Optional) Annotate the frame for visual feedback.
    annotated_frame = frame.copy()
    box_annotator = sv.BoxAnnotator(thickness=2)
    label_annotator = sv.LabelAnnotator(text_thickness=1, text_scale=0.8, text_color=sv.Color.WHITE)
    labels = [
        f"#{tracker_id} {CLASS_NAMES_DICT[class_id]} {conf:0.2f}"
        for conf, class_id, tracker_id in zip(detections.confidence, detections.class_id, detections.tracker_id)
    ]
    annotated_frame = box_annotator.annotate(scene=annotated_frame, detections=detections)
    annotated_frame = label_annotator.annotate(scene=annotated_frame, detections=detections, labels=labels)
    
    return annotated_frame

In [9]:
# --- Process the video ---
sv.process_video(
    source_path=INPUT_VIDEO_PATH,
    target_path=OUTPUT_VIDEO_PATH,
    callback=callback
)

# --- After processing, print unique counts ---
print("Unique counts over the entire video:")
for class_name, track_ids in unique_counts.items():
    print(f"{class_name}: {len(track_ids)}")

Unique counts over the entire video:
dent: 15
scratch: 12


In [10]:
# --- Save the tracking records to a CSV file ---
csv_fields = ["class_name", "unique_count", "unique_tracker_ids"]
with open(CSV_OUTPUT_PATH, mode="w", newline="") as csv_file:
    writer = csv.DictWriter(csv_file, fieldnames=csv_fields)
    writer.writeheader()
    for class_name, track_ids in unique_counts.items():
        writer.writerow({
            "class_name": class_name,
            "unique_count": len(track_ids),
            "unique_tracker_ids": ";".join(str(tid) for tid in track_ids)
        })

print(f"Annotated video saved to: {OUTPUT_VIDEO_PATH}")
print(f"Tracking details saved to: {CSV_OUTPUT_PATH}")

Annotated video saved to: e:\Random Python Scripts\Tata HaxS\supervision\results.mp4
Tracking details saved to: e:\Random Python Scripts\Tata HaxS\supervision\tracking_results.csv
